In [55]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline

from astropy.io import fits 

import os 

import glob 

from astropy import units as u
from astropy.coordinates import SkyCoord

In [100]:
def filesorter(filename, foldername, fitskeyword_to_check, keyword, verbose, science):
    '''
    Function to sort HDI images taken on a single night into the following example directory structure:
    
    20180116/
    ├── bias/
    ├── flats/
    │   ├── dome/
    │       ├── V/
    │       ├── R/
    │       ├── I/
    │       ├── Haon/
    │       └── Haoff/     
    │   └── twilight/
    │       ├── V/
    │       ├── R/
    │       ├── I/
    │       ├── Haon/
    │       └── Haoff/    
    ├── science/
    │   ├── Taurus1/
    │       ├── V/
    │       ├── R/
    │       ├── I/
    │       ├── Haon/
    │       └── Haoff/        
    │   ├── Taurus2/
    │       └── (same filter substructure)/     
    │   ├── Praesepe/
    │       └── (same filter substructure)/     
    │   └── ComaBer/
    │       └── (same filter substructure)/     

    
    Usage:
    filename : should be full path to file location, e.g.,
                /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight4/foo.fits.gz

    science : if RA/Dec string, use a more sophisticated sorting approach based on RA/Dec (RASTRNG, DECSTRNG).
                format of desired target match should be '+04:19:57.9,+28:24:56'
              if None, perform usual sorting. 

    '''
    if os.path.exists(filename):
        pass
    else:
        print(filename + " does not exist or has already been moved.")
        return
    
    header = fits.getheader(filename)
    fits_type = header[keyword]
    
    datadir = '/'.join(filename.split('/')[:-1])
    
    destination = datadir + '/' + foldername
    
    shortfilename = filename.split('/')[-1]
    
    
    if science is None:
        if os.path.exists(destination):
            pass
        else:
            if verbose == True:
                print("Making new directory: " + destination)
            os.mkdir(destination)

        if fits_type == fitskeyword_to_check:
            if verbose == True:
                print("Moving " + filename + " to: " + destination + '/' + shortfilename)
            os.rename(filename, destination + '/' + shortfilename)  
            
    else:
        if os.path.exists(datadir + '/science'):
            pass
        else:
            if verbose == True:
                print("Making new directory: " + datadir + '/science')
            os.mkdir(datadir + '/science')
            
        destination = datadir + '/science/' + foldername
        
        if os.path.exists(destination):
            pass
        else:
            if verbose == True:
                print("Making new directory: " + destination)
            os.mkdir(destination)
            
        desired_ra = science.split(',')[0]
        desired_dec = science.split(',')[1]
        image_ra = header['RASTRNG']
        image_dec = header['DECSTRNG']
        c1 = SkyCoord(desired_ra, desired_dec, unit=(u.hourangle, u.deg), frame='icrs')
        c2 = SkyCoord(image_ra, image_dec, unit=(u.hourangle, u.deg), frame='icrs')
        sep = c1.separation(c2)
        if sep.arcminute < 15.0:
            if verbose == True:
                print("Match Coords: ", desired_ra, desired_dec, ", Image Coords: ", image_ra, image_dec, ", Sep (arcmin): ", sep.arcminute)
                print("Moving " + filename + " to: " + destination + '/' + shortfilename)
            os.rename(filename, destination + '/' + shortfilename)    
        else:
            if verbose == True:
                print("Image coords. do not match criteria.")
        
    return

In [80]:
# use MP's code to check filters:



def verify_hdi_filter(infile,des_filter,verbose=False):
    '''
    verify_filter
    ---------------
    check that the filter we think is being used IS being used
    
    inputs
    ---------
    infile      : (string) filename of image to check
    des_filter  : (string) common filter name (see filter_dict below)
    verbose     : (boolean, default=False) if True, print reason for failure
    
    returns
    ---------
    0,1,2       : (int code) 0=filter matches; 1=filter does not match; 2=one wheel slot not empty
    
    '''
    
    # specify the filter dictionary; hardcoded because it will not change
    filter_dict = {'V':'102',\
               'R':'103',\
               'I':'104',\
               'Ha':'200',\
               'Haoff':'204',\
              'empty':['105','106','107','205','206','207']}
    
    # get the header
    phdr = fits.getheader(infile,0)
    
    # check that a wheel slot is empty
    if ((phdr['FILTER1'] not in filter_dict['empty']) & (phdr['FILTER2'] not in filter_dict['empty']) ):
        
        if verbose:
            print('verify_filter: failed with multiple filters.')
            
        return 2
    
    # check that filter matches the desired wheel position
    elif  (( phdr['FILTER1'] == filter_dict[des_filter]) | ( phdr['FILTER2'] == filter_dict[des_filter]) ):
        return 0
    
    # default to failure mode
    else:
        if verbose:
            print('verify_filter: failed with non-matching filter.')
            
        return 1



# some random testing stuff.

In [23]:
hdr = fits.getheader(bias_frames[0])

In [24]:
hdr['OBJECT']

'bias'

In [48]:
science = '+04:19:57.9,+28:24:56'

In [49]:
        desired_ra = science.split(',')[0]
        desired_dec = science.split(',')[1]

In [53]:
image_dec = hdr['DECSTRNG']

In [54]:
image_ra, image_dec

('+04:19:57.9', '+28:24:56')

In [57]:
c1 = SkyCoord(desired_ra, desired_dec, frame='icrs', unit=(u.hourangle, u.deg))
c2 = SkyCoord(image_ra, image_dec, frame='icrs', unit=(u.hourangle, u.deg))

In [58]:
c1

<SkyCoord (ICRS): (ra, dec) in deg
    ( 64.99125,  28.41555556)>

In [30]:
# test on one image
testimage = '/Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight4/testbias.fits.gz' 
filesorter(testimage, 'testbias', 'BIAS', 'OBSTYPE', verbose=True, science=None)

In [ ]:
# test on single science file.

filesorter('/Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight4/test_sci.fits.gz', \
           'test', 'foo', 'OBSTYPE', verbose=True, science='+12:27:04.3,+26:49:07')

# Sort all biases and flats!

In [97]:
# run on all the bias frames
bias_frames = glob.glob('/Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/c*b00.fits.gz')

for fitsfile in bias_frames:
    filesorter(fitsfile, 'bias', 'BIAS', 'OBSTYPE', verbose=True, science=None)

Making new directory: /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/bias
Moving /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/c8133t0715b00.fits.gz to: /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/bias/c8133t0715b00.fits.gz
Moving /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/c8133t0716b00.fits.gz to: /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/bias/c8133t0716b00.fits.gz
Moving /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/c8133t0717b00.fits.gz to: /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/bias/c8133t0717b00.fits.gz
Moving /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/c8133t0718b00.fits.gz to: /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/bias/c8133t0718b00.fits.gz
Moving /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/c8133t0719b00.fits.gz to: /Volumes/m

In [98]:
# run on all the flat frames
flat_frames = glob.glob('/Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/c*f00.fits.gz')

for fitsfile in flat_frames:
    filesorter(fitsfile, 'flats', 'FLAT', 'OBSTYPE', verbose=True, science=None)

Making new directory: /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/flats
Moving /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/c8133t0001f00.fits.gz to: /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/flats/c8133t0001f00.fits.gz
Moving /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/c8133t0002f00.fits.gz to: /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/flats/c8133t0002f00.fits.gz
Moving /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/c8133t0003f00.fits.gz to: /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/flats/c8133t0003f00.fits.gz
Moving /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/c8133t0004f00.fits.gz to: /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/flats/c8133t0004f00.fits.gz
Moving /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/c8133t0005f00.fits.gz to: /Volu

Moving /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/c8133t0710f00.fits.gz to: /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/flats/c8133t0710f00.fits.gz
Moving /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/c8133t0711f00.fits.gz to: /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/flats/c8133t0711f00.fits.gz
Moving /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/c8133t0712f00.fits.gz to: /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/flats/c8133t0712f00.fits.gz
Moving /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/c8133t0713f00.fits.gz to: /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/flats/c8133t0713f00.fits.gz
Moving /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/c8133t0714f00.fits.gz to: /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/flats/c8133t0714f00.fits.gz


# Sort all science files!

In [101]:
# run on all the "ComaBer" frames
sci_frames = glob.glob('/Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/c*o00.fits.gz')

for fitsfile in sci_frames:
    filesorter(fitsfile, 'ComaBer', 'foo', 'OBSTYPE', verbose=True, science='+12:27:04.3,+26:49:07')

Making new directory: /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/science
Making new directory: /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/science/ComaBer
Image coords. do not match criteria.
Image coords. do not match criteria.
Image coords. do not match criteria.
Image coords. do not match criteria.
Image coords. do not match criteria.
Image coords. do not match criteria.
Image coords. do not match criteria.
Image coords. do not match criteria.
Image coords. do not match criteria.
Image coords. do not match criteria.
Image coords. do not match criteria.
Image coords. do not match criteria.
Image coords. do not match criteria.
Image coords. do not match criteria.
Image coords. do not match criteria.
Image coords. do not match criteria.
Image coords. do not match criteria.
Image coords. do not match criteria.
Image coords. do not match criteria.
Image coords. do not match criteria.
Image coords. do not match criteria.
Image coords. do 

Image coords. do not match criteria.
Image coords. do not match criteria.
Image coords. do not match criteria.
Image coords. do not match criteria.
Image coords. do not match criteria.
Image coords. do not match criteria.
Image coords. do not match criteria.
Image coords. do not match criteria.
Image coords. do not match criteria.
Image coords. do not match criteria.
Image coords. do not match criteria.
Image coords. do not match criteria.
Image coords. do not match criteria.
Image coords. do not match criteria.
Image coords. do not match criteria.
Image coords. do not match criteria.
Image coords. do not match criteria.
Image coords. do not match criteria.
Image coords. do not match criteria.
Image coords. do not match criteria.
Image coords. do not match criteria.
Image coords. do not match criteria.
Image coords. do not match criteria.
Image coords. do not match criteria.
Image coords. do not match criteria.
Image coords. do not match criteria.
Image coords. do not match criteria.
I

Image coords. do not match criteria.
Image coords. do not match criteria.
Image coords. do not match criteria.
Image coords. do not match criteria.
Image coords. do not match criteria.
Image coords. do not match criteria.
Image coords. do not match criteria.
Image coords. do not match criteria.
Image coords. do not match criteria.
Image coords. do not match criteria.
Image coords. do not match criteria.
Image coords. do not match criteria.
Image coords. do not match criteria.
Image coords. do not match criteria.
Image coords. do not match criteria.
Image coords. do not match criteria.
Image coords. do not match criteria.
Image coords. do not match criteria.
Image coords. do not match criteria.
Image coords. do not match criteria.
Image coords. do not match criteria.
Image coords. do not match criteria.
Image coords. do not match criteria.
Image coords. do not match criteria.
Match Coords:  +12:27:04.3 +26:49:07 , Image Coords:  +12:27:04.2 +26:49:18 , Sep (arcmin):  0.184685889139325
M

Match Coords:  +12:27:04.3 +26:49:07 , Image Coords:  +12:27:04.5 +26:49:18 , Sep (arcmin):  0.18868539226804504
Moving /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/c8133t0530o00.fits.gz to: /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/science/ComaBer/c8133t0530o00.fits.gz
Match Coords:  +12:27:04.3 +26:49:07 , Image Coords:  +12:27:04.6 +26:49:18 , Sep (arcmin):  0.19516918285288967
Moving /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/c8133t0531o00.fits.gz to: /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/science/ComaBer/c8133t0531o00.fits.gz
Match Coords:  +12:27:04.3 +26:49:07 , Image Coords:  +12:27:04.7 +26:49:18 , Sep (arcmin):  0.20390040629559855
Moving /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/c8133t0532o00.fits.gz to: /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/science/ComaBer/c8133t0532o00.fits.gz
Match Coords:  +12:27:04.3 +26:49:

Match Coords:  +12:27:04.3 +26:49:07 , Image Coords:  +12:27:04.9 +26:49:18 , Sep (arcmin):  0.22700375857466168
Moving /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/c8133t0566o00.fits.gz to: /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/science/ComaBer/c8133t0566o00.fits.gz
Match Coords:  +12:27:04.3 +26:49:07 , Image Coords:  +12:27:04.9 +26:49:18 , Sep (arcmin):  0.22700375857466168
Moving /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/c8133t0567o00.fits.gz to: /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/science/ComaBer/c8133t0567o00.fits.gz
Match Coords:  +12:27:04.3 +26:49:07 , Image Coords:  +12:27:05.0 +26:49:18 , Sep (arcmin):  0.24083536155374546
Moving /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/c8133t0568o00.fits.gz to: /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/science/ComaBer/c8133t0568o00.fits.gz
Match Coords:  +12:27:04.3 +26:49:

Match Coords:  +12:27:04.3 +26:49:07 , Image Coords:  +12:27:05.3 +26:49:18 , Sep (arcmin):  0.28876939711239324
Moving /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/c8133t0603o00.fits.gz to: /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/science/ComaBer/c8133t0603o00.fits.gz
Match Coords:  +12:27:04.3 +26:49:07 , Image Coords:  +12:27:05.3 +26:49:18 , Sep (arcmin):  0.28876939711239324
Moving /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/c8133t0604o00.fits.gz to: /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/science/ComaBer/c8133t0604o00.fits.gz
Match Coords:  +12:27:04.3 +26:49:07 , Image Coords:  +12:27:05.4 +26:49:18 , Sep (arcmin):  0.3063345588778512
Moving /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/c8133t0605o00.fits.gz to: /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/science/ComaBer/c8133t0605o00.fits.gz
Match Coords:  +12:27:04.3 +26:49:0

Match Coords:  +12:27:04.3 +26:49:07 , Image Coords:  +12:27:05.7 +26:49:18 , Sep (arcmin):  0.36217861912668375
Moving /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/c8133t0640o00.fits.gz to: /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/science/ComaBer/c8133t0640o00.fits.gz
Match Coords:  +12:27:04.3 +26:49:07 , Image Coords:  +12:27:05.9 +26:49:18 , Sep (arcmin):  0.4012970773657661
Moving /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/c8133t0641o00.fits.gz to: /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/science/ComaBer/c8133t0641o00.fits.gz
Match Coords:  +12:27:04.3 +26:49:07 , Image Coords:  +12:27:05.7 +26:49:18 , Sep (arcmin):  0.36217861912668375
Moving /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/c8133t0642o00.fits.gz to: /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/science/ComaBer/c8133t0642o00.fits.gz
Match Coords:  +12:27:04.3 +26:49:0

Match Coords:  +12:27:04.3 +26:49:07 , Image Coords:  +12:27:06.1 +26:49:18 , Sep (arcmin):  0.44146060836012513
Moving /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/c8133t0676o00.fits.gz to: /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/science/ComaBer/c8133t0676o00.fits.gz
Match Coords:  +12:27:04.3 +26:49:07 , Image Coords:  +12:27:06.3 +26:49:18 , Sep (arcmin):  0.48240825598747256
Moving /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/c8133t0677o00.fits.gz to: /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/science/ComaBer/c8133t0677o00.fits.gz
Match Coords:  +12:27:04.3 +26:49:07 , Image Coords:  +12:27:06.1 +26:49:18 , Sep (arcmin):  0.44146060836012513
Moving /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/c8133t0678o00.fits.gz to: /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/science/ComaBer/c8133t0678o00.fits.gz


In [102]:
# run on all the "Praesepe" frames
sci_frames = glob.glob('/Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/c*o00.fits.gz')

for fitsfile in sci_frames:
    filesorter(fitsfile, 'Praesepe', 'foo', 'OBSTYPE', verbose=True, science='+08:40:51.3,+19:34:07')

Making new directory: /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/science/Praesepe
Image coords. do not match criteria.
Image coords. do not match criteria.
Image coords. do not match criteria.
Image coords. do not match criteria.
Image coords. do not match criteria.
Image coords. do not match criteria.
Image coords. do not match criteria.
Image coords. do not match criteria.
Image coords. do not match criteria.
Image coords. do not match criteria.
Image coords. do not match criteria.
Image coords. do not match criteria.
Image coords. do not match criteria.
Image coords. do not match criteria.
Image coords. do not match criteria.
Image coords. do not match criteria.
Image coords. do not match criteria.
Image coords. do not match criteria.
Image coords. do not match criteria.
Image coords. do not match criteria.
Image coords. do not match criteria.
Image coords. do not match criteria.
Image coords. do not match criteria.
Image coords. do not match criteria.
Imag

Image coords. do not match criteria.
Image coords. do not match criteria.
Image coords. do not match criteria.
Image coords. do not match criteria.
Image coords. do not match criteria.
Image coords. do not match criteria.
Image coords. do not match criteria.
Image coords. do not match criteria.
Image coords. do not match criteria.
Image coords. do not match criteria.
Image coords. do not match criteria.
Image coords. do not match criteria.
Image coords. do not match criteria.
Image coords. do not match criteria.
Image coords. do not match criteria.
Image coords. do not match criteria.
Image coords. do not match criteria.
Image coords. do not match criteria.
Image coords. do not match criteria.
Image coords. do not match criteria.
Match Coords:  +08:40:51.3 +19:34:07 , Image Coords:  +08:40:51.7 +19:33:47 , Sep (arcmin):  0.3463951496639651
Moving /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/c8133t0258o00.fits.gz to: /Volumes/mybookduo/Teaching/AST341_Spring2018/

Match Coords:  +08:40:51.3 +19:34:07 , Image Coords:  +08:40:48.4 +19:33:57 , Sep (arcmin):  0.7031680407417304
Moving /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/c8133t0378o00.fits.gz to: /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/science/Praesepe/c8133t0378o00.fits.gz
Match Coords:  +08:40:51.3 +19:34:07 , Image Coords:  +08:40:48.4 +19:33:57 , Sep (arcmin):  0.7031680407417304
Moving /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/c8133t0379o00.fits.gz to: /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/science/Praesepe/c8133t0379o00.fits.gz
Match Coords:  +08:40:51.3 +19:34:07 , Image Coords:  +08:40:48.4 +19:33:57 , Sep (arcmin):  0.7031680407417304
Moving /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/c8133t0380o00.fits.gz to: /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/science/Praesepe/c8133t0380o00.fits.gz
Match Coords:  +08:40:51.3 +19:34:

In [103]:
# run on all the "Taurus2" frames
sci_frames = glob.glob('/Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/c*o00.fits.gz')

for fitsfile in sci_frames:
    filesorter(fitsfile, 'Taurus2', 'foo', 'OBSTYPE', verbose=True, science='+04:19:59.6,+28:25:14')

Making new directory: /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/science/Taurus2
Image coords. do not match criteria.
Image coords. do not match criteria.
Image coords. do not match criteria.
Image coords. do not match criteria.
Image coords. do not match criteria.
Image coords. do not match criteria.
Image coords. do not match criteria.
Image coords. do not match criteria.
Image coords. do not match criteria.
Image coords. do not match criteria.
Image coords. do not match criteria.
Image coords. do not match criteria.
Image coords. do not match criteria.
Image coords. do not match criteria.
Image coords. do not match criteria.
Image coords. do not match criteria.
Image coords. do not match criteria.
Image coords. do not match criteria.
Image coords. do not match criteria.
Image coords. do not match criteria.
Image coords. do not match criteria.
Image coords. do not match criteria.
Image coords. do not match criteria.
Image coords. do not match criteria.
Image

Match Coords:  +04:19:59.6 +28:25:14 , Image Coords:  +04:19:58.2 +28:24:52 , Sep (arcmin):  0.47874980677686996
Moving /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/c8133t0116o00.fits.gz to: /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/science/Taurus2/c8133t0116o00.fits.gz
Match Coords:  +04:19:59.6 +28:25:14 , Image Coords:  +04:19:58.2 +28:24:52 , Sep (arcmin):  0.47874980677686996
Moving /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/c8133t0117o00.fits.gz to: /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/science/Taurus2/c8133t0117o00.fits.gz
Match Coords:  +04:19:59.6 +28:25:14 , Image Coords:  +04:19:58.3 +28:24:52 , Sep (arcmin):  0.46491733298557303
Moving /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/c8133t0118o00.fits.gz to: /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/science/Taurus2/c8133t0118o00.fits.gz
Match Coords:  +04:19:59.6 +28:25:

In [104]:
# run on all the "Taurus1" frames
sci_frames = glob.glob('/Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/c*o00.fits.gz')

for fitsfile in sci_frames:
    filesorter(fitsfile, 'Taurus1', 'foo', 'OBSTYPE', verbose=True, science='+04:32:03.4,+18:14:32')

Making new directory: /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/science/Taurus1
Image coords. do not match criteria.
Image coords. do not match criteria.
Image coords. do not match criteria.
Image coords. do not match criteria.
Image coords. do not match criteria.
Image coords. do not match criteria.
Image coords. do not match criteria.
Image coords. do not match criteria.
Image coords. do not match criteria.
Image coords. do not match criteria.
Match Coords:  +04:32:03.4 +18:14:32 , Image Coords:  +04:31:57.6 +18:16:41 , Sep (arcmin):  2.5531515979278474
Moving /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/c8133t0029o00.fits.gz to: /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/science/Taurus1/c8133t0029o00.fits.gz
Match Coords:  +04:32:03.4 +18:14:32 , Image Coords:  +04:32:01.0 +18:14:19 , Sep (arcmin):  0.6096511195844533
Moving /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/c8133t0030o00.

Match Coords:  +04:32:03.4 +18:14:32 , Image Coords:  +04:32:01.6 +18:14:20 , Sep (arcmin):  0.47186900455618636
Moving /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/c8133t0057o00.fits.gz to: /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/science/Taurus1/c8133t0057o00.fits.gz
Match Coords:  +04:32:03.4 +18:14:32 , Image Coords:  +04:32:01.4 +18:14:20 , Sep (arcmin):  0.5152733392498864
Moving /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/c8133t0058o00.fits.gz to: /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/science/Taurus1/c8133t0058o00.fits.gz
Match Coords:  +04:32:03.4 +18:14:32 , Image Coords:  +04:32:01.4 +18:14:19 , Sep (arcmin):  0.5219687911001456
Moving /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/c8133t0059o00.fits.gz to: /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/science/Taurus1/c8133t0059o00.fits.gz
Match Coords:  +04:32:03.4 +18:14:32

# Sort by filters!


In [83]:
'''
Note to self:
0,1,2       : (int code) 0=filter matches; 1=filter does not match; 2=one wheel slot not empty

'''

filter2names = {'100':"HarrisU",
             '101':"HarrisB",
             '102':"HarrisV", 
             '103':"HarrisR", 
             '104':"HarrisI",
             '105':"empty",
             '106':"empty",
             '107':"empty"}

filter2names = {'200':"6580",
             '201':"6620",
             '202':"6660",
             '203':"6700",
             '204':"6740",
             '205':"empty",
             '206':"empty",
             '207':"empty"}
# smarter.
filter_dict = {'V':'102',\
               'R':'103',\
               'I':'104',\
               'Ha':'200',\
               'Haoff':'204',\
              'empty':['105','106','107','205','206','207']}



In [105]:
# run through the various filters for different fields:

fields = ['Taurus1', 'Taurus2', 'ComaBer', 'Praesepe']

filters = ['V', 'R', 'I', "Ha", "Haoff"]

for field in fields:
    
    for filtername in filters:
            
        imframes = glob.glob('/Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/science/' \
                         + field + '/*fits.gz')
        filterval = filter_dict[filtername]
        
        for fitsfile in imframes:
            verify = verify_hdi_filter(fitsfile, filtername, verbose=False)
            
            if verify == 0:
                if filtername == "Ha" or filtername == "Haoff":
                    filesorter(fitsfile, filtername, filterval, 'FILTER2', verbose=True, science=None)
                else:
                    filesorter(fitsfile, filtername, filterval, 'FILTER1', verbose=True, science=None)

Making new directory: /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/science/Taurus1/V
Moving /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/science/Taurus1/c8133t0029o00.fits.gz to: /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/science/Taurus1/V/c8133t0029o00.fits.gz
Moving /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/science/Taurus1/c8133t0030o00.fits.gz to: /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/science/Taurus1/V/c8133t0030o00.fits.gz
Moving /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/science/Taurus1/c8133t0031o00.fits.gz to: /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/science/Taurus1/V/c8133t0031o00.fits.gz
Moving /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/science/Taurus1/c8133t0032o00.fits.gz to: /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/science/Taurus1/V/c8133

Moving /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/science/Taurus1/c8133t0214o00.fits.gz to: /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/science/Taurus1/R/c8133t0214o00.fits.gz
Moving /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/science/Taurus1/c8133t0215o00.fits.gz to: /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/science/Taurus1/R/c8133t0215o00.fits.gz
Making new directory: /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/science/Taurus1/I
Moving /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/science/Taurus1/c8133t0036o00.fits.gz to: /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/science/Taurus1/I/c8133t0036o00.fits.gz
Moving /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/science/Taurus1/c8133t0037o00.fits.gz to: /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/science/Taurus1/I/c8133

Moving /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/science/Taurus1/c8133t0171o00.fits.gz to: /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/science/Taurus1/Ha/c8133t0171o00.fits.gz
Moving /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/science/Taurus1/c8133t0172o00.fits.gz to: /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/science/Taurus1/Ha/c8133t0172o00.fits.gz
Moving /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/science/Taurus1/c8133t0185o00.fits.gz to: /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/science/Taurus1/Ha/c8133t0185o00.fits.gz
Moving /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/science/Taurus1/c8133t0186o00.fits.gz to: /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/science/Taurus1/Ha/c8133t0186o00.fits.gz
Moving /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/science/Taur

Moving /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/science/Taurus2/c8133t0227o00.fits.gz to: /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/science/Taurus2/R/c8133t0227o00.fits.gz
Moving /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/science/Taurus2/c8133t0228o00.fits.gz to: /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/science/Taurus2/R/c8133t0228o00.fits.gz
Moving /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/science/Taurus2/c8133t0243o00.fits.gz to: /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/science/Taurus2/R/c8133t0243o00.fits.gz
Moving /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/science/Taurus2/c8133t0244o00.fits.gz to: /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/science/Taurus2/R/c8133t0244o00.fits.gz
Moving /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/science/Taurus2/

Moving /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/science/Taurus2/c8133t0251o00.fits.gz to: /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/science/Taurus2/I/c8133t0251o00.fits.gz
Moving /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/science/Taurus2/c8133t0252o00.fits.gz to: /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/science/Taurus2/I/c8133t0252o00.fits.gz
Moving /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/science/Taurus2/c8133t0253o00.fits.gz to: /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/science/Taurus2/I/c8133t0253o00.fits.gz
Moving /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/science/Taurus2/c8133t0254o00.fits.gz to: /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/science/Taurus2/I/c8133t0254o00.fits.gz
Moving /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/science/Taurus2/

...

Moving /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/science/Praesepe/c8133t0391o00.fits.gz to: /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/science/Praesepe/V/c8133t0391o00.fits.gz
Moving /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/science/Praesepe/c8133t0392o00.fits.gz to: /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/science/Praesepe/V/c8133t0392o00.fits.gz
Moving /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/science/Praesepe/c8133t0393o00.fits.gz to: /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/science/Praesepe/V/c8133t0393o00.fits.gz
Moving /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/science/Praesepe/c8133t0407o00.fits.gz to: /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/science/Praesepe/V/c8133t0407o00.fits.gz
Moving /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/science/

Moving /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/science/Praesepe/c8133t0378o00.fits.gz to: /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/science/Praesepe/R/c8133t0378o00.fits.gz
Moving /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/science/Praesepe/c8133t0379o00.fits.gz to: /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/science/Praesepe/R/c8133t0379o00.fits.gz
Moving /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/science/Praesepe/c8133t0380o00.fits.gz to: /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/science/Praesepe/R/c8133t0380o00.fits.gz
Moving /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/science/Praesepe/c8133t0381o00.fits.gz to: /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/science/Praesepe/R/c8133t0381o00.fits.gz
Moving /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/science/

Moving /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/science/Praesepe/c8133t0303o00.fits.gz to: /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/science/Praesepe/I/c8133t0303o00.fits.gz
Moving /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/science/Praesepe/c8133t0304o00.fits.gz to: /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/science/Praesepe/I/c8133t0304o00.fits.gz
Moving /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/science/Praesepe/c8133t0305o00.fits.gz to: /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/science/Praesepe/I/c8133t0305o00.fits.gz
Moving /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/science/Praesepe/c8133t0306o00.fits.gz to: /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/science/Praesepe/I/c8133t0306o00.fits.gz
Moving /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/science/

Moving /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/science/Praesepe/c8133t0498o00.fits.gz to: /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/science/Praesepe/I/c8133t0498o00.fits.gz
Moving /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/science/Praesepe/c8133t0499o00.fits.gz to: /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/science/Praesepe/I/c8133t0499o00.fits.gz
Making new directory: /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/science/Praesepe/Ha
Moving /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/science/Praesepe/c8133t0273o00.fits.gz to: /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/science/Praesepe/Ha/c8133t0273o00.fits.gz
Moving /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/science/Praesepe/c8133t0274o00.fits.gz to: /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/science/Praes

Moving /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/science/Praesepe/c8133t0469o00.fits.gz to: /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/science/Praesepe/Ha/c8133t0469o00.fits.gz
Moving /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/science/Praesepe/c8133t0470o00.fits.gz to: /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/science/Praesepe/Ha/c8133t0470o00.fits.gz
Moving /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/science/Praesepe/c8133t0484o00.fits.gz to: /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/science/Praesepe/Ha/c8133t0484o00.fits.gz
Moving /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/science/Praesepe/c8133t0485o00.fits.gz to: /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/science/Praesepe/Ha/c8133t0485o00.fits.gz
Moving /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight3/scie

In [108]:
# run through the filters for the flats

# N.B. - this requires you to sort the flats into dome or twilight manually based on the logs!

flats = ['dome', 'twilight']

filters = ['V', 'R', 'I', "Ha", "Haoff"]

for flattype in flats:
    
    for filtername in filters:
            
        imframes = glob.glob('/Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight4/flats/' \
                         + flattype + '/*fits.gz')
        filterval = filter_dict[filtername]
        
        for fitsfile in imframes:
            verify = verify_hdi_filter(fitsfile, filtername, verbose=False)
            
            if verify == 0:
                if filtername == "Ha" or filtername == "Haoff":
                    filesorter(fitsfile, filtername, filterval, 'FILTER2', verbose=True, science=None)
                else:
                    filesorter(fitsfile, filtername, filterval, 'FILTER1', verbose=True, science=None)


Making new directory: /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight4/flats/dome/V
Moving /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight4/flats/dome/c8134t0679f00.fits.gz to: /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight4/flats/dome/V/c8134t0679f00.fits.gz
Moving /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight4/flats/dome/c8134t0680f00.fits.gz to: /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight4/flats/dome/V/c8134t0680f00.fits.gz
Moving /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight4/flats/dome/c8134t0681f00.fits.gz to: /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight4/flats/dome/V/c8134t0681f00.fits.gz
Moving /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight4/flats/dome/c8134t0682f00.fits.gz to: /Volumes/mybookduo/Teaching/AST341_Spring2018/WIYNdata/WorkingNight4/flats/dome/V/c8134t0682f00.fits.gz
Moving /Volumes/mybookduo/Te